In [4]:
# Implement an LCEL chain with a memory
# The chain should take the title of a movie and reply with three emojis that represent the movie. 
# (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ").
# using `FewShotPromptTemplate` or `FewShotChatMessagePromptTemplate` to make sure it ""always replies""" with three emojis.
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

#전체 prompt를 메모리에 저장하는 방식 활용
memory = ConversationBufferMemory(
    llm = llm,
    return_messages=True,
    memory_key = "chat_history"
)

#fewshot -> 예제를 모델에게 줘서 모델이 갖춰야할 조건을 학습시킨다.
examples = [
    {
        "movie_name": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie_name": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human","How do you represent {movie_name} movie to emoji?"),
    ("ai","{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = ChatPromptTemplate.from_messages([
        ("human","How do you represent {movie_name} movie to emoji?"),
        ("ai","{answer}")
    ]),
    examples = examples,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a translator AI represent movie to emoji. you can only use three emojies to represent movie"),
    MessagesPlaceholder(variable_name = "chat_history"),
    example_prompt,
    ("human","{answer}"),
])

def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history= load_memory) | prompt | llm

def invoke_chain(answer):
    result = chain.invoke({
        "answer": answer,
    })   
    memory.save_context({"answer": answer},{"output": result.content}) 
    print(result)


In [5]:
invoke_chain("Frozen")
invoke_chain("Parasite")

content='❄️👸🏼⛄'
content='🏠👨\u200d👩\u200d👧\u200d👦💰'


In [6]:
invoke_chain("What is the first movie of my question?")

content='The first movie is "Frozen"'
